# This is a notebook to perform LISCA analysis

Each cell corresponds to a block of code, which can be run by selecting it and pressing shift+enter. The notebook is conceived to be run sequentially. For any errors or questions contact Miguel. Make sure to select the kernel named pyama on the top right. If you cannot find a kernel named pyama, have a look at the confluence page or contact Miguel.

Run the following Cell to make all the imports

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
from nd2reader import ND2Reader
import pandas as pd
from IPython.display import display
import numpy as np
import sys
import time

sys.path.append('/project/ag-moonraedler/MAtienza/pyama/')
from lisca.pipeline import Track

from tqdm import tqdm
import os
from skimage.io import imread
from notebook_viewer import viewers
%matplotlib ipympl

Select the folder where your imaging file is located, as well as the folder where you would like for the output of the analysis to be saved

In [ ]:
data_path = '/project/ag-moonraedler/Nathalie/2022-10-11/'
nd2_file = '2022-10-11-mGL-GA890-L908.nd2'
path_out='.'

You can use this tool to have a look at your images and find out which channel (c) is BF and which which channels are fluorscence.

In [ ]:
viewer = viewers.StackViewer(os.path.join(data_path, nd2_file))

In [ ]:
plt.close(viewer.fig)

In [ ]:
first=0
last=24
positions=range(23, 24+1)
fl_channel=1
bf_channel=0

Choose time points you would like to evaluate

In [ ]:
first_frame=0
last_frame=-1
frames = range(first_frame, last_frame)

If you plan to use cellpose segmentation, use this tool for calibration by checking which parameters work best. Otherwise you may skip this part


In [ ]:
segmentation_viewer = viewers.CellposeViewer(data_path+nd2_file, channel=bf_channel)

In [ ]:
try:
    flow_threshold, diameter, mask_threshold = segmentation_viewer.flow_threshold.value, segmentation_viewer.diameter.value, segmentation_viewer.mask_threshold.value
    plt.close(segmentation_viewer.fig)
except:
    flow_threshold, diameter, mask_threshold = 0.8, 29, -2
    

## Perform LISCA pipeline

Chose a method for segmentation from either "th", the standard pyama method or "cellpose"

In [ ]:
method="th"

Choose the positions you want to evalueate

In [ ]:
n_positions=0
t_0 = time.time()
for fov in positions:
    print(f'Evaluating field of view {fov}')
    
    fovpath=os.path.join(path_out, f'XY{fov}/')
    if not os.path.isdir(fovpath):
        os.mkdir(fovpath)
    tracker = Track(fovpath, data_path, bf_channel, fl_channels, fov=fov, nd2_file=nd2_file, frame_indices=frames)
    tracker.segment(flow_threshold=flow_threshold, mask_threshold=mask_threshold, diameter=diameter, pretrained_model='mdamb231', method=method)
    ##tracker.track(method=method)
    tracker.save_to_pyama(fl_channel)
    n_positions+=1
print(f"Time taken per position: {(time.time()-t_0)/(60*n_positions)} minutes")